In [1]:
import sqlite3
import pandas as pd

DB_PATH = "data/music_data.db"  # adjust if needed

def merge_audio_midi_forwardfill(db_path=DB_PATH):
    # -----------------------------
    # 1. Load tables
    # -----------------------------
    conn = sqlite3.connect(db_path)
    audio_df = pd.read_sql_query("SELECT * FROM audio_features ORDER BY timestamp ASC", conn)
    midi_df  = pd.read_sql_query("SELECT * FROM midi_events ORDER BY timestamp ASC", conn)
    conn.close()

    # -----------------------------
    # 2. Decode MIDI timestamp if necessary
    # -----------------------------
    if pd.api.types.is_object_dtype(midi_df['timestamp']):
        # SQLite may store as bytes
        midi_df['timestamp'] = midi_df['timestamp'].apply(
            lambda x: int.from_bytes(x, 'big') if isinstance(x, bytes) else int(x)
        )
    else:
        midi_df['timestamp'] = midi_df['timestamp'].astype(int)

    # -----------------------------
    # 3. Filter MIDI after first audio timestamp
    # -----------------------------
    first_audio_time = audio_df['timestamp'].min()
    midi_df = midi_df[midi_df['timestamp'] >= first_audio_time]

    # -----------------------------
    # 4. Decode other MIDI columns if bytes
    # -----------------------------
    midi_cols = ['device_id', 'channel', 'note', 'velocity']
    for col in midi_cols:
        if col in midi_df.columns:
            midi_df[col] = midi_df[col].apply(lambda x: int.from_bytes(x, 'big') if isinstance(x, bytes) else x)

    # -----------------------------
    # 5. Convert timestamps to seconds
    # -----------------------------
    audio_df['timestamp_s'] = audio_df['timestamp'] / 1e9
    midi_df['timestamp_s']  = midi_df['timestamp'] / 1e9

    # -----------------------------
    # 6. Sort
    # -----------------------------
    audio_df = audio_df.sort_values('timestamp_s')
    midi_df  = midi_df.sort_values('timestamp_s')

    # -----------------------------
    # 7. Merge asof & forward-fill
    # -----------------------------
    merged_df = pd.merge_asof(
        audio_df,
        midi_df,
        left_on='timestamp_s',
        right_on='timestamp_s',
        direction='backward',  # use last MIDI state
        suffixes=('_audio', '_midi')
    )

    # Forward-fill MIDI state for all audio frames
    for col in midi_cols + ['type', 'program_number']:
        if col in merged_df.columns:
            merged_df[col] = merged_df[col].ffill()

    # Drop redundant columns if present
    merged_df = merged_df.drop(columns=['timestamp_audio', 'timestamp_midi'], errors='ignore')

    return merged_df

# -----------------------------
# Usage
# -----------------------------
merged_df = merge_audio_midi_forwardfill()
print("Merged dataframe shape:", merged_df.shape)
merged_df.head()


Merged dataframe shape: (1537, 18)


<positron-console-cell-1>:67: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


,id_audio,rms_db,rms_delta,centroid,rolloff,flatness,low,mid,high,spectral_flux,onset_strength,timestamp_s,id_midi,device_id,channel,note,velocity,type
0,1,b'2\xf8o\xc2',b'LL\x1c9',3694.013506,10546.875,b'.\x7f\xa0>',b'I\x1c\x1a=',b'#\xa0\xd0<',b't\x0e\x04;',b'yB\xb4<',b'LL\x1c9',1.767069e+09,NaN,NaN,NaN,NaN,NaN,NaN
1,2,b'y\x86k\xc2',b'\xe6&\x9a9',3506.544255,9093.750,b'P\xe7\x9b>',b'\xddNT=',b'9\x12\x16=',b'$8\x17;',b'S\xd0\x8a<',b'\xe6&\x9a9',1.767069e+09,NaN,NaN,NaN,NaN,NaN,NaN
2,3,b'\x1c\xa4j\xc2',b'@\xf3\xf57',3737.161168,9796.875,b'\xabr\xb0>',b'\xd6*)=',b'\xea5#=',b'\x00\xb43;',b'\xab:\x11<',b'@\xf3\xf57',1.767069e+09,NaN,NaN,NaN,NaN,NaN,NaN
3,4,b'^ih\xc2',b'\xb0)\xa28',3953.869335,10218.750,b'\xcb\xa7\xbf>',b'G\xd7\x9f<',b'\x8a\x04;=',b'\n\x11^;',b'\xfd\x91\x19<',b'\xb0)\xa28',1.767069e+09,NaN,NaN,NaN,NaN,NaN,NaN
4,5,b'P\xeai\xc2',b'\xa0\x05]\xb8',4074.491897,10687.500,b'\xe9\x1c\xc9>',b's5\xb6<',b'.c3=',b'{\xab^;',b'\x18)\x05<',0.0,1.767069e+09,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
import sqlite3

# -----------------------------
# Config
# -----------------------------
DB_PATH = "data/clean_music_data.db"

# -----------------------------
# Connect / Create DB
# -----------------------------
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

# -----------------------------
# 1. Audio features table
# -----------------------------
cursor.execute("""
CREATE TABLE IF NOT EXISTS audio_features (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    timestamp INTEGER NOT NULL,          -- nanoseconds
    rms_db REAL,
    rms_delta REAL,
    centroid REAL,
    rolloff REAL,
    flatness REAL,
    low REAL,
    mid REAL,
    high REAL,
    spectral_flux REAL,
    onset_strength REAL
)
""")

# -----------------------------
# 2. MIDI events table
# -----------------------------
cursor.execute("""
CREATE TABLE IF NOT EXISTS midi_events (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    timestamp INTEGER NOT NULL,          -- nanoseconds
    device_id INTEGER,
    channel INTEGER,
    note INTEGER,
    velocity INTEGER,
    cc_number INTEGER,
    cc_value INTEGER,
    program_number INTEGER,
    type TEXT
)
""")

# -----------------------------
# 3. MIDI devices lookup
# -----------------------------
cursor.execute("""
CREATE TABLE IF NOT EXISTS midi_devices (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT UNIQUE
)
""")

# -----------------------------
# Example device insertion
# -----------------------------
devices = [
    "Strymon Cloudburst",
    "Strymon Timeline",
    "Chase Bliss Mood MK II",
    "Chase Bliss Blooper",
    "Chase Bliss Lost And Found",
    "Source Audio EQ2",
    "GT 1000 Core"
]

for device in devices:
    cursor.execute("INSERT OR IGNORE INTO midi_devices (name) VALUES (?)", (device,))

# -----------------------------
# Commit and close
# -----------------------------
conn.commit()
conn.close()

print(f"Clean database created at {DB_PATH} with 3 tables: audio_features, midi_events, midi_devices")


Clean database created at data/clean_music_data.db with 3 tables: audio_features, midi_events, midi_devices


In [27]:
import sqlite3
import pandas as pd
import numpy as np

# -----------------------------
# Paths
# -----------------------------
OLD_DB = "data/music_data.db"
NEW_DB = "data/clean_music_data.db"

# -----------------------------
# 1. Load old audio data
# -----------------------------
old_conn = sqlite3.connect(OLD_DB)
audio_df = pd.read_sql_query("SELECT * FROM audio_features ORDER BY timestamp ASC", old_conn)
old_conn.close()

# -----------------------------
# 2. Decode bytes / clean data
# -----------------------------
def safe_float(val):
    """
    Convert value to float.
    If bytes, try decoding and then float; if fail, return 0.
    If string fails, return 0.
    """
    if isinstance(val, bytes):
        try:
            val = val.decode('utf-8')
        except:
            return 0.0
    try:
        return float(val)
    except:
        return 0.0

# Columns expected to be numeric
numeric_cols = ['rms_db', 'rms_delta', 'centroid', 'rolloff',
                'flatness', 'low', 'mid', 'high', 'spectral_flux', 'onset_strength']

for col in numeric_cols:
    if col in audio_df.columns:
        audio_df[col] = audio_df[col].apply(safe_float)

# Ensure timestamp is int
audio_df['timestamp'] = audio_df['timestamp'].apply(lambda x: int(x) if not isinstance(x, bytes) else int.from_bytes(x, 'big'))

# -----------------------------
# 3. Write to new clean DB
# -----------------------------
new_conn = sqlite3.connect(NEW_DB)
cursor = new_conn.cursor()

for _, row in audio_df.iterrows():
    cursor.execute("""
        INSERT INTO audio_features (
            timestamp, rms_db, rms_delta, centroid, rolloff,
            flatness, low, mid, high, spectral_flux, onset_strength
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, (
        int(row['timestamp']),
        row['rms_db'],
        row['rms_delta'],
        row['centroid'],
        row['rolloff'],
        row['flatness'],
        row['low'],
        row['mid'],
        row['high'],
        row['spectral_flux'],
        row['onset_strength']
    ))

new_conn.commit()
new_conn.close()

print(f"Audio data migrated and cleaned: {len(audio_df)} rows written to {NEW_DB}.")


Audio data migrated and cleaned: 1537 rows written to data/clean_music_data.db.


In [30]:
import sqlite3
import pandas as pd
import numpy as np

NEW_DB = "data/clean_music_data.db"

# MIDI event types
midi_types = ['note_on', 'note_off', 'control_change', 'program_change']

def generate_dummy_midi(audio_df, n_events=1000):
    """Generate dummy MIDI events aligned with audio timestamps."""
    start_ts = audio_df['timestamp'].min()
    end_ts = audio_df['timestamp'].max()

    timestamps = np.linspace(start_ts, end_ts, n_events).astype(int)

    midi_data = []
    for ts in timestamps:
        midi_data.append({
            'timestamp': ts,
            'device_id': np.random.randint(1, 8),          # example: 1-7 devices
            'channel': np.random.randint(0, 16),          # MIDI channels 0-15
            'note': np.random.randint(0, 128),
            'velocity': np.random.randint(0, 128),
            'cc_number': np.random.randint(0, 128),
            'cc_value': np.random.randint(0, 128),
            'program_number': np.random.randint(0, 128),
            'type': np.random.choice(midi_types)
        })
    return pd.DataFrame(midi_data)

# Load audio timestamps
conn = sqlite3.connect(NEW_DB)
audio_df = pd.read_sql_query("SELECT * FROM audio_features ORDER BY timestamp ASC", conn)
conn.close()

# Generate dummy MIDI events
midi_df = generate_dummy_midi(audio_df, n_events=1000)

# Insert into midi_events table
conn = sqlite3.connect(NEW_DB)
cursor = conn.cursor()

for _, row in midi_df.iterrows():
    cursor.execute("""
        INSERT INTO midi_events (
            timestamp, device_id, channel, note, velocity,
            cc_number, cc_value, program_number, type
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, (
        int(row['timestamp']),
        int(row['device_id']),
        int(row['channel']),
        int(row['note']),
        int(row['velocity']),
        int(row['cc_number']),
        int(row['cc_value']),
        int(row['program_number']),
        row['type']
    ))

conn.commit()
conn.close()

print(f"{len(midi_df)} dummy MIDI events generated and written to {NEW_DB}.")


1000 dummy MIDI events generated and written to data/clean_music_data.db.


In [6]:
import sqlite3
import pandas as pd
import numpy as np



DB_PATH = "data/clean_music_data.db"

conn = sqlite3.connect(DB_PATH)

audio_df = pd.read_sql("""
    SELECT *
    FROM audio_features
    ORDER BY timestamp
""", conn)

midi_df = pd.read_sql("""
    SELECT *
    FROM midi_events
    ORDER BY timestamp
""", conn)

conn.close()

audio_df = audio_df.drop(columns=["id"], errors="ignore")
midi_df  = midi_df.drop(columns=["id"], errors="ignore")

audio_cols = [c for c in audio_df.columns if c != "timestamp"]
midi_cols  = [c for c in midi_df.columns  if c != "timestamp"]

print("Audio columns:", audio_cols)
print("MIDI columns:", midi_cols)

timeline = pd.DataFrame({
    "timestamp": np.sort(
        np.unique(
            np.concatenate([
                audio_df["timestamp"].values,
                midi_df["timestamp"].values
            ])
        )
    )
})

timeline = pd.merge_asof(
    timeline,
    audio_df[["timestamp"] + audio_cols],
    on="timestamp",
    direction="backward"
)

midi_merge_df = midi_df[["timestamp"] + midi_cols].copy()
midi_merge_df = midi_merge_df.rename(
    columns={c: f"midi_{c}" for c in midi_cols}
)

timeline = pd.merge_asof(
    timeline.sort_values("timestamp"),
    midi_merge_df.sort_values("timestamp"),
    on="timestamp",
    direction="backward"
)


# Numeric audio and MIDI columns
numeric_audio_cols = timeline[audio_cols].select_dtypes(include=np.number).columns
numeric_midi_cols = timeline[[f"midi_{c}" for c in midi_cols]].select_dtypes(include=np.number).columns

# Flag if **any audio changed**
timeline["audio_changed"] = timeline[numeric_audio_cols].diff().ne(0).any(axis=1)

# Flag if **any MIDI changed**
timeline["midi_changed"] = timeline[numeric_midi_cols].diff().ne(0).any(axis=1)


Audio columns: ['rms_db', 'rms_delta', 'centroid', 'rolloff', 'flatness', 'low', 'mid', 'high', 'spectral_flux', 'onset_strength']
MIDI columns: ['device_id', 'channel', 'note', 'velocity', 'cc_number', 'cc_value', 'program_number', 'type']


In [8]:
timeline

,timestamp,rms_db,rms_delta,centroid,rolloff,flatness,low,mid,high,spectral_flux,...,midi_device_id,midi_channel,midi_note,midi_velocity,midi_cc_number,midi_cc_value,midi_program_number,midi_type,audio_changed,midi_changed
0,1767068566694171904,0.0,0.0,3694.013506,10546.875,0.0,0.0,0.0,0.0,0.0,...,2,14,123.0,82.0,65.0,75.0,103.0,control_change,True,True
1,1767068566881478912,0.0,0.0,3694.013506,10546.875,0.0,0.0,0.0,0.0,0.0,...,4,10,122.0,97.0,86.0,2.0,77.0,control_change,False,True
2,1767068567068785920,0.0,0.0,3694.013506,10546.875,0.0,0.0,0.0,0.0,0.0,...,2,12,83.0,14.0,42.0,100.0,21.0,program_change,False,True
3,1767068567249061888,0.0,0.0,3506.544255,9093.750,0.0,0.0,0.0,0.0,0.0,...,2,12,83.0,14.0,42.0,100.0,21.0,program_change,True,False
4,1767068567256092928,0.0,0.0,3506.544255,9093.750,0.0,0.0,0.0,0.0,0.0,...,7,14,47.0,125.0,53.0,11.0,70.0,program_change,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3215,1767324764822505000,b'\xd0_l\xc2',"b""\x80'6\xb7""",4042.108167,9984.375,b'\xe7\xc9\xd6>',b'hi\x93=',b'\x9a\x1e.=',b'\x82ie;',b'C<\xe7;',...,11,1,NaN,NaN,12.0,110.0,NaN,control_change,True,True
3216,1767324764843676000,b'\xa3\x15m\xc2',b'\xc0j\xbc\xb7',3488.842082,8718.750,b'Ra\xb2>',b'\x8ad\xca=',b'\x1a1\x11=',b'H\xce&;',b'\x1a\x06!<',...,11,1,NaN,NaN,12.0,110.0,NaN,control_change,True,True
3217,1767324764864988000,b'\xfb\x08m\xc2',b'\x00\xcc\xcf5',2734.339661,6421.875,b'vqt>',b'\xdeA\xef=',b'N2\xd5<',"b""\xee'\xc0:""",b'x\xce\x08<',...,11,1,NaN,NaN,12.0,110.0,NaN,control_change,True,True
3218,1767324764886336000,b'HCo\xc2',b'\x90\xd9\x8d\xb8',3737.891818,9609.375,b'&\x18\xbf>',b'\xe7\xbd\x9f=',b'u\xaa\r=',b'\xa3l*;',b'xk7<',...,11,1,NaN,NaN,12.0,110.0,NaN,control_change,True,True


In [2]:
# Features: audio + previous MIDI numeric
feature_cols = audio_cols + [f"midi_{c}" for c in midi_cols]

# Target columns (numeric)
target_cols = ['midi_channel', 'midi_cc_number', 'cc_value']  # adjust exact names in timeline



In [3]:
from sklearn.preprocessing import StandardScaler

scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(timeline[feature_cols])

# Targets can also be scaled if needed (optional)
scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(timeline[target_cols])


ValueError: could not convert string to float: b'_,>\xc2'

array([[1767068566694171904, 0.0, 0.0, 3694.013506162755, 10546.875, 0.0,
        0.0, 0.0, 0.0, 0.0, 0.0, 2, 14, 123, 82, 65, 75, 103,
        'control_change', True, True],
       [1767068566881478912, 0.0, 0.0, 3694.013506162755, 10546.875, 0.0,
        0.0, 0.0, 0.0, 0.0, 0.0, 4, 10, 122, 97, 86, 2, 77,
        'control_change', False, True],
       [1767068567068785920, 0.0, 0.0, 3694.013506162755, 10546.875, 0.0,
        0.0, 0.0, 0.0, 0.0, 0.0, 2, 12, 83, 14, 42, 100, 21,
        'program_change', False, True],
       [1767068567249061888, 0.0, 0.0, 3506.544254667962, 9093.75, 0.0,
        0.0, 0.0, 0.0, 0.0, 0.0, 2, 12, 83, 14, 42, 100, 21,
        'program_change', True, False]], dtype=object)

In [1]:
import sqlite3

DB_PATH = "data/clean_music_data.db"

conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

cursor.execute("DELETE FROM audio_features;")

conn.commit()
conn.close()

print("All rows deleted from audio_features.")


All rows deleted from audio_features.
